In [2]:
day15 = open('inputs/day15.txt').read()
day15 = day15.split('\n')

In [3]:
import re
import z3

s = z3.Solver() # Create solver
x, y = z3.Int("x"), z3.Int("y") # Create int variables

s.add(0 <= x); s.add(x <= 4000000) # Add limits
s.add(0 <= y); s.add(y <= 4000000)

def z3_abs(x):
    return z3.If(x >= 0, x, -x) # If x >= 0, return x, else -x (Return abs)

row_num = 2000000
offset = 10000000
sensors = []
beacons = []
in_range_sensors = []

for sensor in day15:
    s_x, s_y, b_x, b_y = re.findall(r'=((?:.)?\d*)', sensor)
    s_x, s_y, b_x, b_y = int(s_x), int(s_y), int(b_x), int(b_y)
    man_dist = abs(s_x - b_x) + abs(s_y - b_y)
    if ((s_y <= row_num) and (s_y + man_dist) >= row_num) | ((s_y >= row_num) and (s_y - man_dist) <= row_num):
        in_range_sensors.append((s_x, s_y, man_dist))

    sensors.append((s_x,s_y,man_dist))
    beacons.append((b_x,b_y))
            
    s.add(z3_abs(s_x - x) + z3_abs(s_y - y) > man_dist) # Squares must be further from sensor than man_dist) 

import numpy as np

row = np.full((500000000), [0],dtype=int)    
    
for sensor in in_range_sensors:
    distance = sensor[2] - abs(sensor[1]-row_num)    
    centre = sensor[0] + offset
    start = centre-distance

    row[(centre-distance):(centre+distance+1)] = 1
    
#Exclude known beacons
for beacon in beacons:
    if beacon[1] == row_num:
        row[beacon[0]+offset] = 0
      
print(f"Part 1 answer: {row.sum()}")

assert s.check() == z3.sat
model = s.model()
print("Part 2 answer:", model[x].as_long() * 4000000 + model[y].as_long())

Part 1 answer: 4793062
Part 2 answer: 10826395253551


In [128]:
# Find lines that are 1 square outside of each sensors range and store as intercept with Y axis
up_lines = []
down_lines = []
for sensor in sensors:
    sxmin = sensor[0] - sensor[2] -1, sensor[1]
    sxmax = sensor[0] + sensor[2] +1, sensor[1]
    
    up_lines.append(sxmin[1]-sxmin[0])
    up_lines.append(sxmax[1]-sxmax[0])
    
    down_lines.append(sxmin[1]+sxmin[0])
    down_lines.append(sxmax[1]+sxmax[0])
    
# Calculate the coordinates of intersection for each pair of lines
coords = []
for c1, c2 in itertools.product(set(up_lines),set(down_lines)):
    y = (c1 + c2)/2
    x = c2 - y
    if (x >= 0) & (x <= 4000000) & (y >= 0) & (y <= 4000000):
        coords.append((x,y))
        
# For each coordinate, check if inside range of each sensor
for coord in coords:
    out = []
    for sensor in sensors:
        if (abs(sensor[0] - coord[0]) + abs(sensor[1] - coord[1])) > sensor[2]:
            out.append('outside')
            if len(out) == len(sensors): # If outside the range of all sensors
                print(f"Part 2 answer: {int((coord[0]*4000000)+coord[1])}")

Part 2 answer: 10826395253551
